In [46]:
import shutil
from pathlib import Path
import warnings
import random

import numpy as np
import cv2
from PIL import Image
from tqdm.auto import tqdm

In [89]:
dataset_dir = Path("/data/pzndata/augmentor/newdata/teeth0516")
src_img_dir = dataset_dir / 'JPEGImages'
src_gt_dir = dataset_dir / 'SegmentationClass'

dst_img_dir = dataset_dir / 'JPEGImages_Crop'
dst_gt_dir = dataset_dir / 'SegmentationClass_Crop'

In [ ]:
def get_crop(level, img, gt):
    if level not in [70, 80, 90]:
        raise Exception(f"level:{level} not supported.")

    img_size = gt.shape
    crop_size = [int(shape * level / 100) for shape in size]
    start_point_range = [x[0] - x[1] for x in zip(img_size, crop_size)]
    start_point = [int(random.random() * x) for x in start_point_range]
    end_point = [x[0] + x[1] for x in zip(start_point, crop_size)]
    crop_img = img[start_point[0]:end_point[0], start_point[1]:end_point[1]]
    crop_gt = gt[start_point[0]:end_point[0], start_point[1]:end_point[1]]
    
    return crop_img, crop_gt, start_point

In [ ]:
if dst_dataset_dir.exists():
    shutil.rmtree(dst_dataset_dir)
if dst_gt_dir.exists():
    shutil.rmtree(dst_gt_dir)
    
dst_img_dir.mkdir()
dst_gt_dir.mkdir()

for src_img_path in tqdm(list(src_img_dir.glob('*'))):
    src_gt_path = src_gt_dir / (src_img_path.stem + '.png')
    
    if not src_gt_path.exists():
        print(src_gt_path)
        continue
    
    src_img = cv2.imread(str(src_img_path), cv2.IMREAD_UNCHANGED)
    src_gt = cv2.imread(str(src_gt_path), cv2.IMREAD_UNCHANGED)
    
    # Filter gt greater than 3
    src_gt[src_gt>=3] = 0
    
    if src_img.shape[:-1] != src_gt.shape:
        warning.warn(f"Source image shape:{src_img.shape[:-1]} not equal to ground truth shape.")
    
    crop_levels = [70, 80, 90]
    crop_nums = [2, 2, 4]
    
    for level, num in zip(crop_levels, crop_nums):
        for i in range(num):
            crop_img, crop_gt, start_point = get_crop(level, src_img, src_gt)
            crop_name = src_img_path.stem + f"_{level}_" + "_".join(str(x) for x in start_point)
            crop_img_path = dst_img_dir / (crop_name + '.jpg')
            crop_gt_path = dst_gt_dir / (crop_name + '.png')
            cv2.imwrite(str(crop_img_path), crop_img)
            cv2.imwrite(str(crop_gt_path), crop_gt)